In [ ]:
import os
from pathlib import Path
from typing import cast

import pandas as pd
import plotly.express as px
import plotly.io as pio
from IPython.display import display

import modfs.utils.plotting as plot
from modfs.data.result_load import add_derived_columns, load_results

ROOT_PATH = Path().resolve().parent
PATH_DATA = ROOT_PATH / "data"
FIGS_DIR = PATH_DATA / "figs/analysis"
FIGS_DIR.mkdir(exist_ok=True, parents=True)

results_path = PATH_DATA / "run/"

df_original = load_results(results_path)
df_original.loc[
    (df_original["modular_algorithm"] == "cocktail") & (df_original["algorithm"] == "simple"),
    "modular_algorithm",
] = "Cocktail (simple)"
df_original.replace(
    {
        "modular_algorithm": {
            "cocktail": "Cocktail",
            "broadcast": "Broadcast",
            "constraint": "Constraint",
        }
    },
    inplace=True,
)
vals = tuple(str(results_path / f"generic/mixed/duplex/booklet{a}/") + os.sep for a in ["A", "AB"])
df_tmp = df_original[df_original["run_file_path"].str.startswith(vals)].copy()
df_tmp["run_file_path"] = df_tmp["run_file_path"].str.replace(
    str(Path("generic/mixed/duplex")), "comp-orig"
)
df_original = pd.concat([df_original, df_tmp], ignore_index=True)

df_processed = add_derived_columns(
    df_original,
    groups={
        "homogeneous": {"generic/printer_cases"},
        "heterogeneous": {"generic/mixed"},
        "computational": {"computational"},
        "computational-orig": {"comp-orig"},
    },
    gen_subpath="data/gen",
    run_subpath=results_path,
    group_source="run_file_path",
)

# df_normal = plot.add_all_solved(df_processed)
df_normal = df_processed.copy()
df_solved = df_normal[df_normal["solved"]].copy()

figs = {}

In [ ]:
fig = plot.makespan_all_baseline(df_solved, "Constraint")
figs["makespan_compare_all"] = fig
fig.show()


In [ ]:
df_compared = plot.baseline_makespan_compare(*plot.baseline_extract(df_solved, "Constraint"))
df_tmp = df_compared.groupby(["group", "modular_algorithm", "jobs", "modules"], as_index=False)["makespan"].mean()

fig = plot.line_labels(
    df_tmp,
    x="jobs",
    y="makespan",
    line_dash="modular_algorithm",
    color="modules",
    facet_col="group",
    symbol="modules",
    labels={"makespan": "Relative makespan"}
)
fig.add_hline(y=1, line={"color": "red"}, annotation_text="Baseline")
# fig.update_layout(width=1200, height=800)
fig.show()
figs["makespan_compare_by_jobs_and_modules"] = fig

In [ ]:
def print_improvement(df: pd.DataFrame, col_base: str, col_new: str):
    df_means = df.groupby(["modular_algorithm", "group"])["total_time"].mean()
    print((df_means.loc[col_base] / df_means.loc[col_new]).to_string())
    
print_improvement(df_solved, "Constraint", "Cocktail")
fig = plot.runtime_all(
    df_solved,
    title="Execution time of all the solved problems",
)
figs["runtime_solved"] = fig
fig.show()


In [ ]:
df_tmp = df_solved.groupby(["group", "modules", "jobs", "modular_algorithm"], as_index=False)["total_time"].mean()
fig = plot.line_labels(
    df_tmp,
    x="jobs",
    y="total_time",
    line_dash="modular_algorithm",
    # log_y=True,
    facet_col="group",
    color="modules",
    symbol="modules",
    title="Execution time by number of jobs and modules",
)
fig.update_layout(width=1200, height=1000)
fig.show()
figs["runtime_by_jobs_and_modules"] = fig

In [ ]:
df_tmp = df_normal.groupby(["modules", "jobs", "modular_algorithm"], as_index=False)["solved"].mean()
fig = plot.line_labels(
    df_tmp,
    x="jobs",
    y="solved",
    line_dash="modular_algorithm",
    # log_x=True,
    # facet_col="group",
    facet_row="modules",
    facet_col_wrap=1,
    title="Ratio of solved instances by number of jobs and modules",
    width=1200, height=800,
)
# fig.update_layout(automargin=True)
for annotations in fig['layout']['annotations']:
    annotations['textangle'] = 0

fig.show()
figs["solved_by_jobs_and_modules"] = fig

In [ ]:

df_tmp = df_normal.copy()
df_tmp["hardness"] = df_tmp["jobs"] * df_tmp["modules"]
df_tmp = df_tmp.groupby(["hardness", "modular_algorithm"], as_index=False)["solved"].mean()
fig = plot.line_labels(
    df_tmp,
    x="hardness",
    y="solved",
    line_dash="modular_algorithm",
    # facet_col="group",
    # log_x=True,
    title="Ratio of solved instances per hardness (jobs × modules)"
)
fig.show()
figs["solved_by_hardness"] = fig

In [ ]:
for group, df in df_processed.groupby("group"):
    # Plot makespan by jobs
    fig = plot.makespan_by_jobs(df, group, facet_row=None)
    figs[f"makespan_by_jobs_modules_{group}"] = fig
    fig.show()


In [ ]:
for group, df in df_processed.groupby("group"):
    # Plot makespan by jobs
    fig = plot.makespan_by_jobs(df[df["timeout"] == False], group, line_dash=None, facet_row=None)
    figs[f"makespan_by_jobs_modules_no_timeout_{group}"] = fig
    fig.show()    

In [ ]:
for group, df in df_processed.groupby("group"):
    df_temp = df.groupby(["modular_algorithm", "timeout", "jobs"], as_index=False)[
        "makespan"
    ].mean()
    fig = plot.makespan_by_jobs(
        df_temp,
        title=f"Makespan depending on number of jobs of {group}",
        create_df=False,
        color="timeout",
        facet_row=None,
    )
    # figs[f"makespan_by_jobs_{group}"] = fig
    fig.show()


In [ ]:
for group, df in df_processed.groupby("group"):
    df_temp = df.groupby(["modular_algorithm", "timeout", "modules", "jobs"], as_index=False)[
        "total_time"
    ].mean()
    fig = plot.plot_line_defaults(
        data_frame=df_temp,
        x="jobs",
        y="total_time",
        color="modules",
        title=f"Execution time depending on number of jobs of {group}",
        log_y=True,
        facet_row=None,
    )
    
    figs[f"runtime_by_jobs_{group}"] = fig
    fig.show()


In [ ]:
for group, df in df_processed.groupby("group"):
    df_temp = df.groupby(["modular_algorithm", "timeout", "modules"], as_index=False)[
        "total_time"
    ].mean()
    fig = plot.plot_line_defaults(
        data_frame=df_temp,
        x="modules",
        y="total_time",
        color="timeout",
        log_y=True,
        facet_row=None,
        title=f"Execution time depending on number of modules of {group}",
    )
    figs[f"runtime_by_modules_{group}"] = fig
    fig.show()


In [ ]:
FIGS_DIR.mkdir(exist_ok=True)
for path, fig in figs.items():
    print(path)
    pio.write_html(fig, str(FIGS_DIR / f"{path}.html"))
    # fig.update_layout(font={"size": 20})
    # pio.write_image(fig, str(FIGS_DIR / f"{path}.pdf"), width=1600, height=900)

# pio.write_image(fig, '../out/fig.pdf', width=1000, height=800)